In [34]:
import csv
import random
from collections import Counter
from itertools import combinations

In [59]:
def test_loop(boolean_list):
    for i in boolean_list:
        if i is False:
            return False
    return True

def test_all(boolean_list):
    return all(boolean_list)

l = [True,False,True,True,False,False]
r = [True,True]

max_len = 10000
boolean_list = [True if random.random()<0.7 else False for i in range(max_len)]

%timeit test_loop(boolean_list)
%timeit test_all(boolean_list)

125 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
111 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [94]:
input_file = "../data/small1.csv"
support = 5

In [98]:
raw_data = [] # user_id,business_id
with open(input_file,'r') as input:
    readCSV = csv.reader(input, delimiter=',')
    cnt = 0
    for row in readCSV:
        cnt += 1
        if cnt >1:
            raw_data.append(row)

baskets = dict()  # user_id: [business_id1, business_id2, business_id3, ...]
for row in raw_data:
    if row[0] not in baskets:
        baskets[row[0]] = []
    baskets[row[0]].append(row[1])

for user in baskets:
    basket = baskets[user]
    baskets[user] = sorted(basket)
clean_baskets = [baskets[i] for i in baskets]

single_cnt = Counter()
for basket in clean_baskets:
    single_cnt.update(basket)
    
    
max_itemset_size = max([len(basket) for basket in clean_baskets])

candidate_single = sorted([i for i in single_cnt if single_cnt[i]>=support])
former_candidate = candidate_single
print(candidate_single)

candidate_itemsets = {str(i):[] for i in range(1,max_itemset_size+1)}
candidate_itemsets['1'].extend(former_candidate)

for itemset_size in range(2,max_itemset_size+1):
    tmp_itemsets = []
    for basket in clean_baskets:
        if len(basket) < itemset_size:
            continue
        for itemset in combinations(basket,itemset_size):
            itemset = sorted(itemset)
            if itemset_size == 2:
                immediate_subset = [i[0] for i in combinations(itemset,itemset_size-1)]
            else:
                immediate_subset = [sorted(i) for i in combinations(itemset,itemset_size-1)]
            immediate_subset_is_candidate = [i in former_candidate for i in immediate_subset]
            if all(immediate_subset_is_candidate):
                tmp_itemsets.append('_'.join(itemset))
    itemset_cnt = Counter(tmp_itemsets)
    former_candidate = [i.split('_') for i in itemset_cnt if itemset_cnt[i]>=support]
    candidate_itemsets[str(itemset_size)].extend(former_candidate)

['100', '101', '102', '97', '98', '99']


In [96]:
candidate_itemsets

{'1': ['100', '101', '102', '97', '98', '99'],
 '2': [['100', '101'],
  ['101', '102'],
  ['102', '98'],
  ['101', '99'],
  ['97', '99'],
  ['97', '98'],
  ['98', '99']],
 '3': [['97', '98', '99']],
 '4': [],
 '5': [],
 '6': [],
 '7': [],
 '8': [],
 '9': [],
 '10': []}

In [82]:
def market_basket(input_file,support,verbose=False):
    raw_data = [] # user_id,business_id
    with open(input_file,'r') as input:
        readCSV = csv.reader(input, delimiter=',')
        cnt = 0
        for row in readCSV:
            cnt += 1
            if cnt >1:
                raw_data.append(row)

    item_frequency = dict()
    for row in raw_data:
        if row[1] not in item_frequency:
            item_frequency[row[1]] = 1
        else:
            item_frequency[row[1]] += 1

    frequent_item = []
    for item in item_frequency:
        if item_frequency[item] >= support:
            frequent_item.append(item)


    if verbose is True:
        print("\nFrequent items: ",frequent_item)

    baskets = dict()  # user_id: [business_id1, business_id2, business_id3, ...]
    for row in raw_data:
        if row[0] not in baskets:
            baskets[row[0]] = []
        baskets[row[0]].append(row[1])

    for user in baskets:
        basket = baskets[user]
        baskets[user] = sorted(basket)


    pair_cnt = dict()
    frequent_pair = []
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-1):
            for j in range(i+1, basket_len):
                pair = basket[i] + '_' + basket[j]
                if pair not in pair_cnt:
                    pair_cnt[pair] = 0
                pair_cnt[pair] += 1

    for pair in pair_cnt:
        if pair_cnt[pair] >= support:
            double_itemset = pair.split('_')
            frequent_pair.append(double_itemset)
    if verbose is True:
        print("\nFrequent pairs: ", frequent_pair)

    frequent_pair_string = []
    for pair in frequent_pair:
        pair_string = '_'.join(pair)
        frequent_pair_string.append(pair_string)

    triple_cnt = dict()
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-2):
            for j in range(i+1, basket_len-1):
                for k in range(j+1, basket_len):
                    p1 = basket[i] + '_' + basket[j]
                    p2 = basket[j] + '_' + basket[k]
                    p3 = basket[i] + '_' + basket[k]
                    if p1 in frequent_pair_string and p2 in frequent_pair_string and p3 in frequent_pair_string:
                        triple =  basket[i] + '_' + basket[j] + '_' + basket[k]
                        if triple not in triple_cnt:
                            triple_cnt[triple] = 0
                        triple_cnt[triple] += 1
    
    frequent_triple = []
    for triple in triple_cnt:
        if triple_cnt[triple] >= support:
            frequent_triple.append(triple.split("_"))
    if verbose is True:
        print("\nFrequent triples: ", frequent_triple)


    candidate_after_triple = []
    for i in frequent_triple:
        candidate_after_triple.extend(i)    

    candidate_after_triple = sorted(list(set(candidate_after_triple)))
    

    four_cnt = dict()
    frequent_triple_string = []
    for i in frequent_triple:
        frequent_triple_string.append('_'.join(i))
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-3):
            for j in range(i+1, basket_len-2):
                for k in range(j+1, basket_len-1):
                    for m in range(k+1, basket_len):
                        p1 = basket[i] + '_' + basket[j] + '_' + basket[k]
                        p2 = basket[j] + '_' + basket[k] + '_' + basket[m]
                        p3 = basket[i] + '_' + basket[j] + '_' + basket[m]
                        p4 = basket[j] + '_' + basket[k] + '_' + basket[m]
                        if p1 in frequent_triple_string and p2 in frequent_triple_string and p3 in frequent_triple_string:
                            if p4 in frequent_triple_string: 
                                four =  basket[i] + '_' + basket[j] + '_' + basket[k] + '_' + basket[m]
                                if four not in four_cnt:
                                    four_cnt[four] = 0
                                four_cnt[four] += 1
    
    
    frequent_four = []
    for four in four_cnt:
        if four_cnt[four] >= support:
            frequent_four.append(four.split("_"))
            
    
    five_cnt = []
    
    
    return {"singleton": frequent_item, "pair": frequent_pair, "triple": frequent_triple, "four":frequent_four}

In [63]:
%timeit  market_basket(input_file,support)

659 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [93]:
market_basket(input_file,5,True)


Frequent items:  ['100', '98', '101', '102', '99', '97']

Frequent pairs:  [['100', '101'], ['101', '102'], ['102', '98'], ['101', '99'], ['97', '99'], ['97', '98'], ['98', '99']]

Frequent triples:  [['97', '98', '99']]


{'singleton': ['100', '98', '101', '102', '99', '97'],
 'pair': [['100', '101'],
  ['101', '102'],
  ['102', '98'],
  ['101', '99'],
  ['97', '99'],
  ['97', '98'],
  ['98', '99']],
 'triple': [['97', '98', '99']],
 'four': []}

In [65]:
%timeit  result = market_basket("../data/small2.csv",6)

216 ms ± 8.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
result = market_basket("../data/small2.csv",6)

In [69]:
market_basket("../data/small2.csv",10,True)


Frequent items:  ['107', '106', '103', '97', '105', '100', '101', '102', '99', '116', '111', '112', '115', '108', '104', '98', '110', '113', '114', '109']

Frequent pairs:  [['100', '103'], ['100', '105'], ['100', '106'], ['100', '107'], ['100', '97'], ['103', '105'], ['103', '106'], ['103', '107'], ['103', '97'], ['105', '106'], ['105', '107'], ['105', '97'], ['106', '107'], ['106', '97'], ['107', '97'], ['100', '101'], ['100', '102'], ['100', '99'], ['101', '102'], ['101', '103'], ['101', '106'], ['101', '107'], ['101', '97'], ['101', '99'], ['102', '103'], ['102', '106'], ['102', '107'], ['102', '111'], ['102', '112'], ['102', '97'], ['102', '99'], ['103', '111'], ['103', '112'], ['103', '99'], ['106', '111'], ['106', '99'], ['107', '111'], ['107', '112'], ['107', '99'], ['112', '99'], ['97', '99'], ['100', '104'], ['100', '108'], ['101', '104'], ['101', '108'], ['103', '104'], ['103', '108'], ['104', '106'], ['104', '107'], ['104', '108'], ['104', '97'], ['106', '108'], ['107', '1

{'singleton': ['107',
  '106',
  '103',
  '97',
  '105',
  '100',
  '101',
  '102',
  '99',
  '116',
  '111',
  '112',
  '115',
  '108',
  '104',
  '98',
  '110',
  '113',
  '114',
  '109'],
 'pair': [['100', '103'],
  ['100', '105'],
  ['100', '106'],
  ['100', '107'],
  ['100', '97'],
  ['103', '105'],
  ['103', '106'],
  ['103', '107'],
  ['103', '97'],
  ['105', '106'],
  ['105', '107'],
  ['105', '97'],
  ['106', '107'],
  ['106', '97'],
  ['107', '97'],
  ['100', '101'],
  ['100', '102'],
  ['100', '99'],
  ['101', '102'],
  ['101', '103'],
  ['101', '106'],
  ['101', '107'],
  ['101', '97'],
  ['101', '99'],
  ['102', '103'],
  ['102', '106'],
  ['102', '107'],
  ['102', '111'],
  ['102', '112'],
  ['102', '97'],
  ['102', '99'],
  ['103', '111'],
  ['103', '112'],
  ['103', '99'],
  ['106', '111'],
  ['106', '99'],
  ['107', '111'],
  ['107', '112'],
  ['107', '99'],
  ['112', '99'],
  ['97', '99'],
  ['100', '104'],
  ['100', '108'],
  ['101', '104'],
  ['101', '108'],
  ['103